# Matrix Multiplication



$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Where:
- $Q$ is the Query matrix
- $K$ is the Key matrix  
- $V$ is the Value matrix
- $d_k$ is the dimension of the key vectors
- $\text{softmax}$ normalizes the attention weights